In [1]:
import argparse

from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )  # cuda:number or cpu
        self.data = "ctrp"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    # 0,  # Cell
    1  # Drug
]

In [5]:
def nihgcn_new(
    cell_exprs,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
):

    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)
    
    val_labels = sampler.test_data[sampler.test_mask] 

    if len(np.unique(val_labels)) < 2:
        print(f"Target {target_index} skipped: Validation set has only one class.")
        return None, None 
    
    model = nihgcn(
        sampler.train_data,
        cell_exprs=cell_exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        evaluate_fun,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    )
    true_data, predict_data = opt()
    return true_data, predict_data

In [ ]:
from joblib import Parallel, delayed
from tqdm import tqdm

n_kfold = 1
n_jobs = 50  # 並列数

def process_iteration(dim, target_index, seed, args):
    """各反復処理をカプセル化した関数"""
    if dim:
        if drug_sum[target_index] < 10:
            return None, None
    else:
        if cell_sum[target_index] < 10:
            return None, None

    fold_results = []
    for fold in range(n_kfold):
        true_data, predict_data = nihgcn_new(
            cell_exprs=exprs,
            drug_finger=drug_finger,
            res_mat=res,
            null_mask=null_mask,
            target_dim=dim,
            target_index=target_index,
            evaluate_fun=roc_auc,
            args=args,
            seed=seed,
        )
        fold_results.append((true_data, predict_data))
    
    return fold_results

# 並列処理の実行
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()

for dim in target_dim:
    # 全タスクを事前に生成
    tasks = [
        (dim, target_index, seed, args)
        for seed, target_index in enumerate(np.arange(res.shape[dim]))
    ]
    
    # 並列実行（プログレスバー付き）
    results = Parallel(n_jobs=n_jobs, verbose=0, prefer="threads")(
        delayed(process_iteration)(*task) for task in tqdm(tasks, desc=f"Processing dim {dim}")
    )
    
    # 結果の結合
    for fold_results in results:
        if fold_results is None:
            continue
        for true_data, predict_data in fold_results:
            true_data_s = pd.concat(
                [true_data_s, translate_result(true_data)], 
                ignore_index=True,
                copy=False  # メモリ節約のため
            )
            predict_data_s = pd.concat(
                [predict_data_s, translate_result(predict_data)], 
                ignore_index=True,
                copy=False
            )


  0%|          | 0/460 [00:00<?, ?it/s]

epoch:   0 loss:0.701933 auc:0.4364
epoch:  20 loss:0.352359 auc:0.7432
epoch:  40 loss:0.328279 auc:0.7194
epoch:  60 loss:0.305437 auc:0.7375
epoch:  80 loss:0.289149 auc:0.7416
epoch: 100 loss:0.263085 auc:0.7661
epoch: 120 loss:0.249578 auc:0.7873
epoch: 140 loss:0.241921 auc:0.7894
epoch: 160 loss:0.235987 auc:0.7917
epoch: 180 loss:0.233814 auc:0.7874
epoch: 200 loss:0.229469 auc:0.7931
epoch: 220 loss:0.226948 auc:0.7836
epoch: 240 loss:0.225776 auc:0.7897
epoch: 260 loss:0.226728 auc:0.7938
epoch: 280 loss:0.223379 auc:0.7844
epoch: 300 loss:0.222068 auc:0.7902
epoch: 320 loss:0.220435 auc:0.7784
epoch: 340 loss:0.219821 auc:0.7808
epoch: 360 loss:0.218804 auc:0.7741
epoch: 380 loss:0.222659 auc:0.7672
epoch: 400 loss:0.218242 auc:0.7668
epoch: 420 loss:0.217032 auc:0.7623
epoch: 440 loss:0.216398 auc:0.7616
epoch: 460 loss:0.217021 auc:0.7597
epoch: 480 loss:0.215731 auc:0.7610
epoch: 500 loss:0.217204 auc:0.7673
epoch: 520 loss:0.215185 auc:0.7596
epoch: 540 loss:0.218967 auc

  0%|          | 1/460 [01:05<8:17:49, 65.08s/it]

Fit finished.
epoch:   0 loss:0.701984 auc:0.5547
epoch:  20 loss:0.352088 auc:0.6797
epoch:  40 loss:0.329061 auc:0.7344
epoch:  60 loss:0.304319 auc:0.7734
epoch:  80 loss:0.282181 auc:0.7500
epoch: 100 loss:0.262204 auc:0.7461
epoch: 120 loss:0.254971 auc:0.6875
epoch: 140 loss:0.241920 auc:0.7070
epoch: 160 loss:0.237726 auc:0.7188
epoch: 180 loss:0.233068 auc:0.6875
epoch: 200 loss:0.229345 auc:0.6992
epoch: 220 loss:0.228145 auc:0.7109
epoch: 240 loss:0.225752 auc:0.7148
epoch: 260 loss:0.226740 auc:0.7188
epoch: 280 loss:0.223110 auc:0.7070
epoch: 300 loss:0.225259 auc:0.7422
epoch: 320 loss:0.221134 auc:0.7188
epoch: 340 loss:0.222694 auc:0.7148
epoch: 360 loss:0.219204 auc:0.7070
epoch: 380 loss:0.233077 auc:0.4453
epoch: 400 loss:0.220069 auc:0.7148
epoch: 420 loss:0.217619 auc:0.7227
epoch: 440 loss:0.216772 auc:0.7227
epoch: 460 loss:0.217935 auc:0.7148
epoch: 480 loss:0.216083 auc:0.7109
epoch: 500 loss:0.218948 auc:0.6484
epoch: 520 loss:0.215853 auc:0.7188
epoch: 540 los

  0%|          | 2/460 [02:03<7:49:02, 61.45s/it]

...

Fit finished.
epoch:   0 loss:0.700788 auc:0.5484
epoch:  20 loss:0.353819 auc:0.4144
epoch:  40 loss:0.329603 auc:0.4695
epoch:  60 loss:0.307256 auc:0.5432
epoch:  80 loss:0.282801 auc:0.6296
epoch: 100 loss:0.264206 auc:0.6380
epoch: 120 loss:0.251043 auc:0.6177
epoch: 140 loss:0.242456 auc:0.6072
epoch: 160 loss:0.237889 auc:0.6092
epoch: 180 loss:0.233225 auc:0.5993
epoch: 200 loss:0.231019 auc:0.5476
epoch: 220 loss:0.228754 auc:0.5572
epoch: 240 loss:0.226532 auc:0.5643
epoch: 260 loss:0.223934 auc:0.5475
epoch: 280 loss:0.224205 auc:0.5508
epoch: 300 loss:0.221524 auc:0.5756
epoch: 320 loss:0.224183 auc:0.5419
epoch: 340 loss:0.219887 auc:0.5838
epoch: 360 loss:0.219745 auc:0.5691
epoch: 380 loss:0.219293 auc:0.5618
epoch: 400 loss:0.218700 auc:0.5632
epoch: 420 loss:0.217973 auc:0.5677
epoch: 440 loss:0.216993 auc:0.5740
epoch: 460 loss:0.218030 auc:0.5666
epoch: 480 loss:0.216015 auc:0.5613
epoch: 500 loss:0.220685 auc:0.5840
epoch: 520 loss:0.217319 auc:0.5710
epoch: 540 los

  6%|▌         | 27/460 [24:53<6:41:25, 55.63s/it]

Fit finished.
epoch:   0 loss:0.700177 auc:0.4867
epoch:  20 loss:0.352363 auc:0.6405
epoch:  40 loss:0.328152 auc:0.6149
epoch:  60 loss:0.306174 auc:0.6062
epoch:  80 loss:0.282035 auc:0.5824
epoch: 100 loss:0.263607 auc:0.5903
epoch: 120 loss:0.249497 auc:0.5828
epoch: 140 loss:0.242191 auc:0.5863
epoch: 160 loss:0.235911 auc:0.5634
epoch: 180 loss:0.231779 auc:0.5698
epoch: 200 loss:0.229508 auc:0.5856
epoch: 220 loss:0.227724 auc:0.5708
epoch: 240 loss:0.224820 auc:0.5812
epoch: 260 loss:0.224270 auc:0.5690
epoch: 280 loss:0.222014 auc:0.5779
epoch: 300 loss:0.223991 auc:0.5751
epoch: 320 loss:0.220326 auc:0.5838
epoch: 340 loss:0.219553 auc:0.5771
epoch: 360 loss:0.220888 auc:0.5714
epoch: 380 loss:0.218527 auc:0.5799
epoch: 400 loss:0.218257 auc:0.5845
epoch: 420 loss:0.218145 auc:0.5818
epoch: 440 loss:0.228828 auc:0.5798
epoch: 460 loss:0.216667 auc:0.5773
epoch: 480 loss:0.215742 auc:0.5678
epoch: 500 loss:0.215559 auc:0.5776
epoch: 520 loss:0.215227 auc:0.5831
epoch: 540 los

  6%|▌         | 28/460 [25:52<6:46:50, 56.51s/it]

Fit finished.
epoch:   0 loss:0.703859 auc:0.5253
epoch:  20 loss:0.353549 auc:0.4625
epoch:  40 loss:0.329039 auc:0.5050
epoch:  60 loss:0.308196 auc:0.5665
epoch:  80 loss:0.281911 auc:0.5827
epoch: 100 loss:0.263302 auc:0.5844
epoch: 120 loss:0.251083 auc:0.5846
epoch: 140 loss:0.245373 auc:0.5873
epoch: 160 loss:0.235991 auc:0.5951
epoch: 180 loss:0.231591 auc:0.5893
epoch: 200 loss:0.229444 auc:0.5938
epoch: 220 loss:0.231527 auc:0.6036
epoch: 240 loss:0.225550 auc:0.6017
epoch: 260 loss:0.225379 auc:0.5990
epoch: 280 loss:0.222469 auc:0.5968
epoch: 300 loss:0.222794 auc:0.5956
epoch: 320 loss:0.220334 auc:0.5929
epoch: 340 loss:0.221223 auc:0.5909
epoch: 360 loss:0.218918 auc:0.5904
epoch: 380 loss:0.221324 auc:0.6058
epoch: 400 loss:0.218295 auc:0.5908
epoch: 420 loss:0.217328 auc:0.5958
epoch: 440 loss:0.217476 auc:0.5990
epoch: 460 loss:0.218488 auc:0.5991
epoch: 480 loss:0.215801 auc:0.6082
epoch: 500 loss:0.216944 auc:0.6052
epoch: 520 loss:0.215034 auc:0.6031
epoch: 540 los

In [ ]:
true_data_s.to_csv(f"new_drug_true_{args.data}.csv")
predict_data_s.to_csv(f"new_drug_pred_{args.data}.csv")